# ERP analysis tutorial



In [1]:
import os

import numpy as np
import pandas as pd

from mne import read_evokeds
from mne import grand_average
from mne.viz import plot_compare_evokeds

from meeg_tools.time_frequency import read_evokeds_from_path, get_erp_peak_measures, get_erp_measures_from_cross_condition_data

from matplotlib import pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

%matplotlib qt

/Users/weian/.conda/envs/eeg-workshop/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.0, the latest is 0.6.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


## Read the data

In [2]:
# Change this path below to the path where the pipeline saved the (.fif.gz)  ERP files!
erp_data_path = '/Volumes/t7-ssd/TRIPLET-EPOCH-2'


# Read the data
erp_H = read_evokeds_from_path(erp_data_path, pattern='day1_h')
erp_L = read_evokeds_from_path(erp_data_path, pattern='day1_l')


erp_all_conditions = read_evokeds_from_path(erp_data_path, pattern='day1')

In [8]:
# Check the number of files for each condition
len(erp_H), len(erp_L)


(180, 180)

In [7]:
# Check the number of files for all conditions
len(erp_all_conditions)

1800

## Compare ERP between the predictable and unpredictable stimuli

In [ ]:
# Inspect grand average power per condition (H, L)
erp_H_grand_average = grand_average(erp_H)
erp_L_grand_average = grand_average(erp_L)

# Ignore subset of channels (['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF8', 'AF4'])
ch_names =  [ch for ch in erp_H_grand_average.info['ch_names'] if ch not in ['Fp1', 'Fp2', 
                                                                               'AF7', 'AF3',
                                                                               'AFz', 'AF8', 'AF4']]

erp_H_grand_average = erp_H_grand_average.pick_channels(ch_names)#.apply_baseline((-0.25, 0.0),)
erp_L_grand_average = erp_L_grand_average.pick_channels(ch_names)#.apply_baseline((-0.25, 0.0),)

In [ ]:
# Inspect scalp topographies with the plot_topomap method

# Here we display topomaps from -200 ms, to 750 ms with 50 ms step size.
times = np.arange(-0.2, 0.8, 0.05) # from -200 ms to 750 ms with 50 ms step size
# You can use fewer times with explicitly defining the time points in a list:
#times = [0.1, 0.2, 0.5]
erp_H_grand_average.plot_topomap(nrows=2, ncols=10, times=times, average=None)

In [ ]:
erp_L_grand_average.plot_topomap(nrows=2, ncols=10, times=times, average=None)

In [ ]:
# This will plot ERPs for each channel, click on a channel to expand the plot
erp_H_grand_average.copy().plot_topo(title='Grand Average ERP')

In [ ]:
# This will plot ERPs for each channel, click on a channel to expand the plot
erp_L_grand_average.copy().plot_topo(title='Grand Average ERP')

In [ ]:
# Plot with selected channels
picks = ['F7', 'F5', 'F1', 'F4', 'F8', 'F2']

tmin = -0.25 # plot data from 250 ms prior to stimulus onset
tmax = 0.75  # plot data to 750 ms after to stimulus onset
baseline = (-0.25, 0.0) # apply baseline from 250 ms prior to stimulus onset to 0.0

# To remove baseline uncomment this line below
#baseline = (None, None)


evokeds = dict(triplet_H=erp_H_grand_average.copy().crop(tmin=tmin, tmax=tmax).apply_baseline(baseline),
               triplet_L=erp_L_grand_average.copy().crop(tmin=tmin, tmax=tmax).apply_baseline(baseline))

plot_compare_evokeds(evokeds,
                     combine='mean', # 'median'
                     picks=picks)

# with all channels
plot_compare_evokeds(evokeds,
                     combine='mean') # 'median'

In [9]:
# combined grand average (all subjects and all conditions)
# Plot with selected channels
picks = ['F7', 'F5', 'F1', 'F4', 'F8', 'F2']

erp_all_conditions_average = grand_average(erp_all_conditions)

erp_all_conditions_average = erp_all_conditions_average.copy().apply_baseline((-0.25, 0.0),)

plot_compare_evokeds(evokeds=erp_all_conditions_average,
                     picks=picks,
                    combine='mean') # 'median'

Identifying common channels ...
combining channels using "mean"


[<Figure size 1600x1200 with 1 Axes>]

## Peak latency and amplitude


picks: the list of channel names to calculate peak latency and amplitude # required


mode='pos': finds the peak with a positive voltage (ignores negative voltages)

mode='neg': finds the peak with a negative voltage (ignores positive voltages)

mode='abs': finds the peak with the largest absolute voltage regardless of sign (positive or negative)



combine='mean': aggregates channels defined by picks using mean. (can be 'median' or 'std')

combine=None: does not aggregate channels but calculates peak latency and amplitude separately for each channel

In [39]:
cross_condition_measures = get_erp_peak_measures(erp=erp_all_conditions_average,
                                                 tmin=0.12,
                                                 tmax=0.22,
                                                 mode='neg',
                                                 picks=["F7"], # the list of channel names
                                                combine=None) # use 'mean' to aggregate channels, otherwise it will calculate peak latency and amplitude separately for the channels

cross_condition_measures

,fid,ch_name,tmin,tmax,mode,peak_latency,peak_amplitude
0,Grand average (n = 1800),F7,0.12,0.22,neg,0.122,-1.21559


In [40]:
peak_measures = get_erp_measures_from_cross_condition_data(erp_arrays=erp_all_conditions,
                                                           cross_condition_data=cross_condition_measures,
                                                           interval_in_seconds=0.05)

C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\103_R_Day1_H_1 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\103_R_Day1_H_2 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\103_R_Day1_H_3 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\103_R_Day1_H_4 :No negative values encountered. Usin

C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\118_B_Day1_L_4 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\118_B_Day1_L_5 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\11_S_Day1_L_1 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\120_R_Day1_H_4 :No negative values encountered. Using

C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\135_S_Day1_H_4 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\135_S_Day1_H_5 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\135_S_Day1_L_1 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\135_S_Day1_L_3 :No negative values encountered. Usin

C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\148_B_Day1_H_4 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\148_B_Day1_L_5 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\149_B_Day1_L_2 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\14_S_Day1_H_1 :No negative values encountered. Using

C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\164_B_Day1_L_3 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\164_B_Day1_L_4 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\164_B_Day1_L_5 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\170_B_Day1_H_1 :No negative values encountered. Usin

...

C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\44_E_Day1_L_4 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\44_E_Day1_L_5 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\46_E_Day1_H_3 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\46_E_Day1_H_4 :No negative values encountered. Using de

C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\58_E_Day1_H_3 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\58_E_Day1_H_4 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\58_E_Day1_H_5 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\58_E_Day1_L_1 :No negative values encountered. Using de

C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\65_E_Day1_H_1 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\65_E_Day1_H_2 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\65_E_Day1_H_3 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\65_E_Day1_H_4 :No negative values encountered. Using de

C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\71_S_Day1_L_3 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\71_S_Day1_L_4 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\71_S_Day1_L_5 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\75_S_Day1_H_2 :No negative values encountered. Using de

C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\90_L_Day1_H_3 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\90_L_Day1_H_4 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\94_L_Day1_H_1 :No negative values encountered. Using default mode.
C:\Users\teodora.vekony\BML-MEMO LAB Dropbox\Lab_workspace\Experiments_Projects\Recent_projects\clinical_brain_projects\TMS_rewiring\Preprocessed_data\epochs_asrt\erp\Day1\TRIPLET-EPOCH\94_L_Day1_H_5 :No negative values encountered. Using de

In [42]:
peak_measures

,fid,ch_name,tmin,tmax,mode,peak_latency,peak_amplitude,mean_amplitude
0,100_R_Day1_H_1,F7,0.072,0.172,neg,0.114,-2.828919,-1.813199
1,100_R_Day1_H_2,F7,0.072,0.172,neg,0.120,-3.275072,-1.946540
2,100_R_Day1_H_3,F7,0.072,0.172,neg,0.124,-0.723871,-0.435369
3,100_R_Day1_H_4,F7,0.072,0.172,neg,0.118,-3.036356,-1.609299
4,100_R_Day1_H_5,F7,0.072,0.172,neg,0.122,-2.472926,-1.658594
5,100_R_Day1_L_1,F7,0.072,0.172,neg,0.120,-1.622244,-0.839849
6,100_R_Day1_L_2,F7,0.072,0.172,neg,0.110,-3.095055,-1.710296
7,100_R_Day1_L_3,F7,0.072,0.172,neg,0.138,-1.441413,-0.731897
8,100_R_Day1_L_4,F7,0.072,0.172,neg,0.128,-3.303951,-2.439133
9,100_R_Day1_L_5,F7,0.072,0.172,neg,0.104,-2.334711,-1.278687


In [13]:
# This is a convenient way to exctract triplet information from the "fid" column values
# E.g. 10_S_Day1_H  with this command we extract -> 10, S, Day1, H 
# E.g. 10_S_Day1_1_H modify from 3 to 4 to extract epochs information as well
#peak_measures.loc[:, 'epoch'] = peak_measures['fid'].str.split('_', 4, expand=True)[3]
#peak_measures.loc[:, 'triplet'] = peak_measures['fid'].str.split('_', 4, expand=True)[4]

#print(peak_measures['fid'].str.split('_', 3, expand=True))

#peak_measures['fid'] = peak_measures['fid'].str.split("\\")[0][-1]

peak_measures.loc[:, 'triplet'] = peak_measures['fid'].str.split('_', 3, expand=True)[3]


In [14]:
peak_measures

,fid,ch_name,tmin,tmax,mode,peak_latency,peak_amplitude,mean_amplitude,epoch,triplet
0,100_R_Day1_H_1,F7,0.072,0.172,neg,0.114,-2.828919,-1.813199,1,H
1,100_R_Day1_H_2,F7,0.072,0.172,neg,0.120,-3.275072,-1.946540,2,H
2,100_R_Day1_H_3,F7,0.072,0.172,neg,0.124,-0.723871,-0.435369,3,H
3,100_R_Day1_H_4,F7,0.072,0.172,neg,0.118,-3.036356,-1.609299,4,H
4,100_R_Day1_H_5,F7,0.072,0.172,neg,0.122,-2.472926,-1.658594,5,H
5,100_R_Day1_L_1,F7,0.072,0.172,neg,0.120,-1.622244,-0.839849,1,L
6,100_R_Day1_L_2,F7,0.072,0.172,neg,0.110,-3.095055,-1.710296,2,L
7,100_R_Day1_L_3,F7,0.072,0.172,neg,0.138,-1.441413,-0.731897,3,L
8,100_R_Day1_L_4,F7,0.072,0.172,neg,0.128,-3.303951,-2.439133,4,L
9,100_R_Day1_L_5,F7,0.072,0.172,neg,0.104,-2.334711,-1.278687,5,L


In [15]:
peak_measures.groupby(['ch_name', 'triplet'])['mean_amplitude'].describe()

count      mean       std       min       25%       50%  \
ch_name triplet                                                            
F7      H        900.0 -0.066011  1.467155 -2.867848 -1.006593 -0.479491   
        L        900.0 -0.153296  1.490068 -2.989243 -1.099297 -0.546736   

                      75%        max  
ch_name triplet                       
F7      H        0.811436  10.774301  
        L       -0.028813   7.476895

In [ ]:
peak_measures.to_csv(os.path.join(erp_data_path, 'peak_measures.csv'), index=False)